In [ ]:
from surprise import accuracy
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate, GridSearchCV

In [ ]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Крос-валідація для оцінки моделі SVD
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# Крос-валідація для оцінки моделі SVDpp
algo = SVDpp()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
nmf = NMF()
# Крос-валідація для оцінки моделі NMF
results_nmf = cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# Параметри для пошуку для SVD та SVD++
param_grid = {
    'n_factors': [50, 100, 150],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}

In [ ]:
# Застосуємо GridSearchCV для пошуку оптимальних параметрів для SVD
grid_search_svd = GridSearchCV(SVD, param_grid, measures=['RMSE'], cv=3)
grid_search_svd.fit(data)

# Найкращі параметри для SVD
print("Best parameters for SVD:", grid_search_svd.best_params['rmse'])
print("Best RMSE for SVD:", grid_search_svd.best_score['rmse'])

In [ ]:
# SVD++
grid_search_svdpp = GridSearchCV(SVDpp, param_grid, measures=['RMSE'], cv=2, n_jobs=-1)
grid_search_svdpp.fit(data)

# Найкращі параметри для SVD++
print("Best parameters for SVD++:", grid_search_svdpp.best_params['rmse'])
print("Best RMSE for SVD++:", grid_search_svdpp.best_score['rmse'])

In [ ]:
# Для NMF
param_grid_nmf = {
    'n_factors': [50, 100, 150],  # кількість латентних факторів
    'n_epochs': [20, 30, 40],     # кількість епох
    'reg_pu': [0.02, 0.1],        # регуляризація для факторів користувачів
    'reg_qi': [0.02, 0.1]         # регуляризація для факторів предметів
}

grid_search_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['RMSE'], cv=3)
grid_search_nmf.fit(data)

# Найкращі параметри для NMF
print("Best parameters for NMF:", grid_search_nmf.best_params['rmse'])
print("Best RMSE for NMF:", grid_search_nmf.best_score['rmse'])

In [ ]:
#Best parameters for SVD: {'n_factors': 150, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.1}
best_svd = SVD(n_factors=150, n_epochs=40, lr_all=0.005, reg_all=0.1)
best_svd.fit(data.build_full_trainset())

# Оцінка точності на тестових даних
testset = data.build_full_trainset().build_testset()
predictions = best_svd.test(testset)

# Підрахуємо RMSE на тестових даних
rmse = accuracy.rmse(predictions)
print("Test RMSE for the best SVD model:", rmse)

In [ ]:
#Best parameters for SVD++: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
best_svd = SVDpp(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02)
best_svd.fit(data.build_full_trainset())

# Оцінка точності на тестових даних
testset = data.build_full_trainset().build_testset()
predictions = best_svd.test(testset)

# Підрахуємо RMSE на тестових даних
rmse = accuracy.rmse(predictions)
print("Test RMSE for the best SVD model:", rmse)

SVD показала найкращий результат за RMSE = 0.9263
SVD++ показав RMSE на етапі крос-валідації = 0.9199
NMF показала найгірші результати за RMSE = 0.9594

після навчання накращі результати показала модель SVD++
Для SVD: RMSE = 0.7829
Для SVD++: RMSE = 0.7400
Тож модель SVD++ виявилася більш ефективною на тестовому наборі